In [157]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.io import arff
import python_scripts.data_processing as dp

import sklearn

In [158]:
"""
data = arff.loadarff("../data/3year.arff")
df = pd.DataFrame(data[0])
X_train, X_test, y_train, y_test = dp.get_train_test(df)

y_test = pd.Series(y_test[0])
y_train = pd.Series(y_train[0])
"""

'\ndata = arff.loadarff("../data/3year.arff")\ndf = pd.DataFrame(data[0])\nX_train, X_test, y_train, y_test = dp.get_train_test(df)\n\ny_test = pd.Series(y_test[0])\ny_train = pd.Series(y_train[0])\n'

In [159]:
data = arff.loadarff("../data/3year.arff")
df = pd.DataFrame(data[0])
y = df.iloc[:, -1]
df = df.iloc[:, :64]

In [160]:
def as_discrete(col):
    n = len(col)
    new_col = [0] * n
    for i in range(n):
        if col[i] == b"0":
            new_col[i] = 0
        else:
            new_col[i] = 1
    return pd.Series(new_col)


y = as_discrete(y)

In [161]:
def med_impute(df, y):
    # remove columns with more than 40% values being null
    thd1 = df.shape[0] * 0.4
    cols = df.columns[df.isnull().sum() < thd1]
    df = df[cols]

    # remove rows with more than 50% values being null
    thd2 = df.shape[1] * 0.5
    y = y[df.isnull().sum(axis=1) <= thd2]
    df = df[df.isnull().sum(axis=1) <= thd2]

    # median imputation for null values
    for column in df.columns:
        df[column] = df[column].fillna(df[column].median())

    return df, y


df, y = med_impute(df, y)

In [162]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.2, random_state=1000
)
sum(y_test == 1)

101

In [163]:
# Initialize KNN classifier with k=5
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9509757258448358


In [165]:
from sklearn.preprocessing import MinMaxScaler


def normalise(df):
    scaler = MinMaxScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

    return X_scaled


df1 = normalise(df)

In [166]:
def drop_high_corr(df, threshold=0.7):

    correlation_matrix = df.corr()
    high_cor = []
    dropped_features = []

    # Iterate through the correlation matrix to find highly correlated pairs
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                if correlation_matrix.columns[j] != correlation_matrix.columns[i]:
                    high_cor.append(
                        [
                            correlation_matrix.columns[i],
                            correlation_matrix.columns[j],
                            correlation_matrix.iloc[i, j],
                        ]
                    )

    # Iterate through the list of highly correlated pairs
    for pair in high_cor:
        feature1, feature2, correlation = pair

        # Check if either of the features in the pair has already been dropped
        if feature1 not in dropped_features and feature2 not in dropped_features:
            # Check if the feature exists in the DataFrame before attempting to drop it
            if feature2 in df.columns:
                # Drop one of the correlated features from the dataset
                # Here, we arbitrarily choose to drop the second feature in the pair
                df.drop(feature2, axis=1, inplace=True)
                dropped_features.append(feature2)
            else:
                print(f"Feature '{feature2}' not found in the DataFrame.")

    return df


df1 = drop_high_corr(df1, threshold=0.7)
df1.info

<bound method DataFrame.info of           Attr5     Attr9    Attr13    Attr15    Attr21    Attr24    Attr26  \
0      0.945548  0.003252  0.112756  0.184936  0.000070  0.255539  0.024190   
1      0.945550  0.003799  0.112753  0.184955  0.000207  0.253054  0.024176   
2      0.945557  0.002998  0.112736  0.185056  0.000060  0.253371  0.024149   
3      0.945549  0.003056  0.112737  0.185152  0.000058  0.254762  0.024143   
4      0.945545  0.003445  0.112764  0.184937  0.000077  0.252395  0.024189   
...         ...       ...       ...       ...       ...       ...       ...   
10496  0.945548  0.003004  0.112734  0.185118  0.000070  0.252407  0.024145   
10497  0.945551  0.002916  0.112734  0.185437  0.000061  0.252253  0.024133   
10498  0.945539  0.003122  0.112709  0.184704  0.000069  0.251848  0.024111   
10499  0.945549  0.003004  0.112733  0.185430  0.000062  0.252427  0.024137   
10500  0.945546  0.004507  0.112733  0.186036  0.000079  0.252476  0.024133   

         Attr27    

In [167]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    df1, y, test_size=0.2, random_state=148
)

# Initialize KNN classifier with k=5
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train1, y_train1)

# Predict on the test set
y_pred1 = knn.predict(X_test1)

# Calculate accuracy
accuracy = accuracy_score(y_test1, y_pred1)
print("Accuracy:", accuracy)

Accuracy: 0.9576392194193242
